## **1. Import Library**

In [1]:
# --- Core Libraries ---
# Untuk membaca file PDF
!pip install -q pdfplumber

# Untuk membaca file DOCX
!pip install -q python-docx

# --- NLP & AI Libraries ---
# Untuk Ekstraksi Kata Kunci (KeyBERT) dan Model Dasar
!pip install -q keybert sentence-transformers transformers[torch] accelerate # Use [torch] for GPU support in transformers

!pip install -q nltk

!pip install -q scikit-learn

# (Yake optional if you don't plan to use it)
# print("Installing yake...")
# !pip install -q yake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.6 MB/s eta 0:00:00


## **2. Setup dan Konfigurasi**

In [2]:
import pdfplumber
import docx
import os
import time
import json # For saving results
import numpy as np
from google.colab import drive
from keybert import KeyBERT
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch # To check for GPU availability
from sklearn.cluster import KMeans
from collections import Counter
import itertools
import nltk

# --- Global Settings & Configuration ---
# [cite_start]Path ke folder dataset Anda di Google Drive [cite: 323, 330]
DATASET_FOLDER = "/content/drive/MyDrive/PaperTA"
# Path untuk menyimpan hasil JSON
RESULTS_JSON_PATH = "/content/drive/MyDrive/PaperTA_results.json"
GRAPH_JSON_PATH = "/content/drive/MyDrive/PaperTA_graph_data.json"
SENTENCE_RELATIONS_JSON_PATH = "/content/drive/MyDrive/PaperTA_sentence_relations.json"

# Batas karakter untuk input model (pencegahan error GPU/memori)
# Sesuaikan jika perlu, 4000 char ~ 1000-1200 tokens
MAX_CHARS_FOR_MODEL = 3000
# Ambang batas kemiripan untuk visualisasi hubungan
SIMILARITY_THRESHOLD = 0.50
SENTENCE_SIMILARITY_THRESHOLD = 0.50

NLTK_DATA_PATH = "/content/nltk_data"
os.makedirs(NLTK_DATA_PATH, exist_ok=True)

if NLTK_DATA_PATH not in nltk.data.path:
    nltk.data.path.append(NLTK_DATA_PATH)
print(f"NLTK Data Path: {nltk.data.path}")

print(f"\nMemastikan data NLTK (punkt) ada di {NLTK_DATA_PATH}...")
try:
    # Coba cari di path spesifik kita
    nltk.data.find('tokenizers/punkt', paths=[NLTK_DATA_PATH])
    print("✅ Data NLTK (punkt) sudah tersedia di lokasi kustom.")
except LookupError:
    print(f"⚠️ Data NLTK (punkt) tidak ditemukan, mencoba mengunduh ke {NLTK_DATA_PATH}...")
    try:
        nltk.download('punkt', download_dir=NLTK_DATA_PATH, quiet=True) # Download ke folder spesifik
        # Verifikasi ulang setelah download
        nltk.data.find('tokenizers/punkt', paths=[NLTK_DATA_PATH])
        print("✅ Data NLTK (punkt) berhasil diunduh dan diverifikasi.")
    except Exception as download_error:
        print(f"❌ FATAL ERROR: Gagal mengunduh data NLTK (punkt). Error: {download_error}")
        raise download_error # Hentikan jika download gagal

# Cek ketersediaan GPU
DEVICE = 0 if torch.cuda.is_available() else -1 # 0 for first GPU, -1 for CPU
DEVICE_NAME = torch.cuda.get_device_name(0) if DEVICE == 0 else "CPU"
print(f"💡 Menggunakan Perangkat: {DEVICE_NAME}")

# --- Mount Google Drive ---
print("\nMenghubungkan Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True) # force_remount helps avoid stale mounts
    print("✅ Google Drive terhubung.")
    # Verifikasi path dataset
    if not os.path.exists(DATASET_FOLDER):
        print(f"⚠️ Peringatan: Folder dataset '{DATASET_FOLDER}' tidak ditemukan di Google Drive Anda.")
    else:
        print(f"✅ Folder dataset ditemukan di '{DATASET_FOLDER}'.")
except Exception as e:
    print(f"❌ Gagal menghubungkan Google Drive: {e}")

NLTK Data Path: ['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/content/nltk_data']

Memastikan data NLTK (punkt) ada di /content/nltk_data...
⚠️ Data NLTK (punkt) tidak ditemukan, mencoba mengunduh ke /content/nltk_data...
✅ Data NLTK (punkt) berhasil diunduh dan diverifikasi.
💡 Menggunakan Perangkat: Tesla T4

Menghubungkan Google Drive...
Mounted at /content/drive
✅ Google Drive terhubung.
✅ Folder dataset ditemukan di '/content/drive/MyDrive/PaperTA'.


## **3. Fungsi Helper Pemrosesan Dokumen (PDF, DOCX, BERT, BART, Embedding)**

In [3]:
import nltk
import os
import pdfplumber
import docx
import re  # Import Regex untuk pembersihan teks tingkat lanjut

# --- Lokasi Eksplisit untuk NLTK Data ---
NLTK_DATA_PATH = "/content/nltk_data"
os.makedirs(NLTK_DATA_PATH, exist_ok=True)

# --- Helper Function: Smart Text Cleaner ---
def clean_and_locate_content(text):
    """
    Fungsi canggih untuk membersihkan metadata jurnal dan mencari awal konten nyata.
    """
    # 1. Gabungkan pemenggalan kata di akhir baris (misal: "commu-\nnication" -> "communication")
    text = re.sub(r'-\n', '', text)

    # 2. Hapus spasi berlebih dan newline aneh
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. Cari titik mulai yang relevan (Biasanya "Abstract" atau "Introduction")
    # Kita cari posisi kata "Abstract" (case insensitive)
    match = re.search(r'(?:abstract|introduction|background)', text, re.IGNORECASE)

    if match:
        # Jika ketemu, kita ambil teks MULAI dari situ
        start_index = match.start()
        # Kita ambil agak banyak (misal 4000 char) dari titik mulai tsb
        relevant_text = text[start_index:]
    else:
        # Jika tidak ada kata Abstract, kita coba skip 300-500 karakter pertama
        # (asumsi itu header metadata)
        relevant_text = text[500:]

    return relevant_text

# --- Fungsi NLP ---

def split_into_sentences(text):
    """Memisahkan teks panjang menjadi kalimat menggunakan NLTK."""
    if not text or not isinstance(text, str): return []
    try:
        # Failsafe NLTK Punkt (seperti sebelumnya)
        if NLTK_DATA_PATH not in nltk.data.path: nltk.data.path.append(NLTK_DATA_PATH)
        try: nltk.data.find('tokenizers/punkt', paths=[NLTK_DATA_PATH])
        except LookupError: nltk.download('punkt', download_dir=NLTK_DATA_PATH, quiet=True)

        sentences = nltk.sent_tokenize(text)
        return [s.strip() for s in sentences if len(s.strip()) > 10]
    except Exception: return []

def extract_text_from_pdf(file_path):
    """Mengekstrak teks PDF dengan strategi pembersihan."""
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            # Hanya ambil 2-3 halaman pertama (biasanya Abstrak & Intro ada di sini)
            # Mengambil seluruh halaman seringkali membuang resource untuk Reference list
            pages_to_extract = pdf.pages[:3]
            for page in pages_to_extract:
                page_text = page.extract_text(x_tolerance=1, y_tolerance=3)
                if page_text: text += page_text + " "
        return text
    except Exception as e:
        print(f"⚠️ Gagal memproses PDF {os.path.basename(file_path)}: {e}")
        return None

def extract_text_from_docx(file_path):
    text = ""
    try:
        doc = docx.Document(file_path)
        # Ambil 50 paragraf pertama saja
        paragraphs = [para.text.strip() for para in doc.paragraphs[:50] if para.text.strip()]
        return " ".join(paragraphs)
    except Exception as e:
        print(f"⚠️ Gagal memproses DOCX {os.path.basename(file_path)}: {e}")
        return None

# --- FUNGSI UTAMA YANG DIPERBAIKI ---
MAX_CHARS_FOR_MODEL = 3500 # Sedikit diperbesar untuk konteks

def generate_summary_bart(text, summarizer_instance):
    """
    Menghasilkan ringkasan abstrakif yang berwawasan.
    Menggunakan teknik 'Smart Localization' untuk menghindari metadata.
    """
    if not text or len(text) < 100: return "Teks terlalu pendek."

    try:
        # 1. Bersihkan dan cari konten inti (Abstract/Intro)
        cleaned_text = clean_and_locate_content(text)

        # 2. Potong sesuai batas model
        truncated_text = cleaned_text[:MAX_CHARS_FOR_MODEL]

        # 3. Generate Summary dengan parameter yang dituning agar lebih mengalir
        summary = summarizer_instance(
            truncated_text,
            max_length=200,   # Biarkan agak panjang agar informatif
            min_length=60,    # Paksa minimal 60 token agar tidak terlalu pendek
            do_sample=False,  # Deterministic beam search (lebih akurat untuk fakta)
            length_penalty=2.0, # Mendorong model membuat ringkasan yang tuntas
            num_beams=4       # Beam search width
        )
        return summary[0]['summary_text']

    except RuntimeError as e:
        if "CUDA" in str(e): print(f"❌ Error CUDA saat ringkasan."); return "Gagal (CUDA Error)."
        return "Gagal (Runtime Error)."
    except Exception:
        return "Gagal membuat ringkasan."

def extract_keywords_bert(text, kw_model_instance, top_n=10, ngram_range=(1, 2)): # Ubah ngram ke 1-2 agar lebih fokus
    """Ekstraksi kata kunci dengan input yang sudah dibersihkan."""
    if not text or len(text) < 50: return []
    try:
        # Gunakan juga teks yang sudah dibersihkan agar keyword tidak ambil metadata
        cleaned_text = clean_and_locate_content(text)[:MAX_CHARS_FOR_MODEL]

        keywords = kw_model_instance.extract_keywords(
            cleaned_text,
            keyphrase_ngram_range=ngram_range,
            stop_words='english',
            use_maxsum=True, nr_candidates=20, top_n=top_n
        )
        # Filter keyword pendek/angka
        return [kw for kw, score in keywords if len(kw) > 3 and not kw.isdigit()]
    except Exception: return []

def generate_embeddings(text, embedding_model_instance):
    if not text or len(text) < 50: return None
    try:
        # Embedding juga mendapat manfaat dari teks bersih
        cleaned_text = clean_and_locate_content(text)[:MAX_CHARS_FOR_MODEL*2]
        return embedding_model_instance.encode(cleaned_text, convert_to_numpy=True)
    except Exception: return None

print("✅ Fungsi helper (Smart NLP Updated) siap.")

✅ Fungsi helper (Smart NLP Updated) siap.


## **4. Load Model NLP (KeyBERT, BART, dan SentenceTransformer)**

In [4]:
print("⏳ Memuat model KeyBERT (all-MiniLM-L6-v2)...")
# Menggunakan model standar yang bagus untuk semantic tasks
kw_model = KeyBERT(model='all-MiniLM-L6-v2')
print("✅ Model KeyBERT berhasil dimuat.")

print("⏳ Memuat model Peringkasan (BART - distilbart-cnn-12-6)...")
# Model ini bagus untuk ringkasan berita/artikel umum, cukup ringan
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=DEVICE)
print("✅ Model Peringkasan berhasil dimuat.")

print("⏳ Memuat model Embedding (all-MiniLM-L6-v2)...")
# Menggunakan model yang sama dengan KeyBERT untuk efisiensi & konsistensi embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda' if DEVICE == 0 else 'cpu')
print("✅ Model Embedding berhasil dimuat.")

⏳ Memuat model KeyBERT (all-MiniLM-L6-v2)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


✅ Model Peringkasan berhasil dimuat.
⏳ Memuat model Embedding (all-MiniLM-L6-v2)...
✅ Model Embedding berhasil dimuat.


## **5. Proses Utama: Ekstraksi, Analisis, dan Penyimpanan Hasil**

In [5]:
print(f"🚀 Memulai Proses Inti dari folder: {DATASET_FOLDER}\n" + "="*50)

# --- Inisialisasi Variabel ---
results = {}
embeddings_map = {}
filenames_list = []

try:
    all_files = [f for f in os.listdir(DATASET_FOLDER) if f.lower().endswith(('.pdf', '.docx'))]
    total_files = len(all_files)
    if total_files == 0:
        print(f"⚠️ Tidak ada file PDF atau DOCX yang ditemukan.")
    else:
        print(f"🔍 Menemukan {total_files} file PDF/DOCX untuk diproses.")
except FileNotFoundError:
    print(f"❌ FATAL ERROR: Folder dataset tidak ditemukan. Hentikan eksekusi.")
    all_files = []

file_counter = 0
start_time = time.time()

print("\n--- TAHAP 1: Ekstraksi & Analisis NLP per Dokumen ---")
for filename in all_files:
    # ... (Salin seluruh loop 'for filename in all_files:' Anda dari Cell 5) ...
    # ... (Pastikan semua kode ekstraksi teks, keyword, summary, embedding ada di sini) ...
    file_counter += 1
    print(f"\n[{file_counter}/{total_files}]--- Processing: {filename} ---")
    file_path = os.path.join(DATASET_FOLDER, filename)
    results[filename] = {'text_length': 0, 'keywords': [], 'summary': ''}
    full_text = None
    start_extract = time.time()
    if filename.lower().endswith(".pdf"):
        full_text = extract_text_from_pdf(file_path)
    elif filename.lower().endswith(".docx"):
        full_text = extract_text_from_docx(file_path)
    extract_duration = time.time() - start_extract

    if full_text:
        text_len = len(full_text)
        results[filename]['text_length'] = text_len
        print(f"📄 Teks diekstrak ({text_len:,} chars) dalam {extract_duration:.2f} dtk.")

        kw_start = time.time()
        keywords = extract_keywords_bert(full_text, kw_model)
        results[filename]['keywords'] = keywords
        kw_duration = time.time() - kw_start
        print(f"🔑 Kata Kunci ({len(keywords)}): {keywords} ({kw_duration:.2f} dtk)")

        sum_start = time.time()
        summary_text = generate_summary_bart(full_text, summarizer)
        results[filename]['summary'] = summary_text
        sum_duration = time.time() - sum_start
        summary_display = summary_text[:150]+"..." if not summary_text.startswith("Gagal") else summary_text
        print(f"📝 Ringkasan: {summary_display} ({sum_duration:.2f} dtk)")

        emb_start = time.time()
        embedding_vector = generate_embeddings(full_text, embedding_model)
        emb_duration = time.time() - emb_start
        if embedding_vector is not None:
             embeddings_map[filename] = embedding_vector
             filenames_list.append(filename)
             results[filename]['embedding'] = embedding_vector.tolist()
             print(f"🧠 Embedding dibuat (dim: {embedding_vector.shape[0]}) ({emb_duration:.2f} dtk)")
        else:
             results[filename]['embedding'] = None
             print(f"❌ Embedding Gagal.")
    else:
        print(f"❌ Ekstraksi Teks Gagal.")
        results[filename]['error'] = 'Text extraction failed'
        results[filename]['embedding'] = None

# --- Finalisasi & Penyimpanan Hasil TAHAP 1 ---
end_time_tahap1 = time.time()
print("\n" + "="*50 + f"\n✅ TAHAP 1 Selesai!")
print(f"⏱️ Total Waktu T1: {end_time_tahap1 - start_time:.2f} detik")
print(f"📊 Jumlah File Diproses: {file_counter}")
print(f"📈 Jumlah File dengan Embedding: {len(embeddings_map)}")
print(f"\n💾 Menyimpan hasil detail ke: {RESULTS_JSON_PATH}")
try:
    # (Salin kode penyimpanan JSON 'results_to_save' Anda dari Cell 5)
    results_to_save = {}
    for fname, data in results.items():
        results_to_save[fname] = data.copy()
        if fname in embeddings_map:
             if embeddings_map[fname] is not None and isinstance(embeddings_map[fname], np.ndarray):
                  results_to_save[fname]['embedding'] = embeddings_map[fname].tolist()
             else:
                  results_to_save[fname]['embedding'] = None
        else:
             results_to_save[fname]['embedding'] = None
    with open(RESULTS_JSON_PATH, 'w') as f:
        json.dump(results_to_save, f, indent=2)
    print(f"✅ Hasil berhasil disimpan.")
except Exception as e:
    print(f"❌ Gagal menyimpan hasil ke JSON: {e}")

🚀 Memulai Proses Inti dari folder: /content/drive/MyDrive/PaperTA
🔍 Menemukan 5 file PDF/DOCX untuk diproses.

--- TAHAP 1: Ekstraksi & Analisis NLP per Dokumen ---

[1/5]--- Processing: Comprehensive Comparative Study of.pdf ---
📄 Teks diekstrak (17,530 chars) dalam 4.03 dtk.
🔑 Kata Kunci (10): ['predictive performance', 'labels predicted', 'mlc taxonomy', 'class classification', 'methods datasets', 'mlc multitask', 'multi label', 'learning methods', 'benchmark datasets', 'mlc label'] (15.48 dtk)
📝 Ringkasan:  Multi-label classification (MLC) has recently received increasing interest from the machine learning community . This work provides a comprehensive e... (1.61 dtk)
🧠 Embedding dibuat (dim: 384) (0.01 dtk)

[2/5]--- Processing: Classification of mental illnesses on social media using RoBERTa.pdf ---


📄 Teks diekstrak (13,272 chars) dalam 1.95 dtk.
🔑 Kata Kunci (10): ['pandemic', 'work pandemic', 'social mental', 'anxiety bipolar', 'anxiety world', 'suicide prevention', 'emotions online', 'illness social', 'depression2 attributed', 'communication depression2'] (4.98 dtk)
📝 Ringkasan:  Social media has become the primary mode of communication for most or depression . This can be attributed to a myriad people suffering of reasons inc... (0.81 dtk)
🧠 Embedding dibuat (dim: 384) (0.01 dtk)

[3/5]--- Processing: A Computational Approach to Understand Mental Health from RedditKnowledge-Aware Multitask Learning Framework.pdf ---
📄 Teks diekstrak (16,906 chars) dalam 2.39 dtk.
🔑 Kata Kunci (10): ['disease user', 'symptoms post', 'posts comments', 'twitter talklife', 'share mental', 'persons heart', 'gender collect', 'identifies symptoms', 'analyzing gender', 'comments annotated'] (5.04 dtk)
📝 Ringkasan:  Social media platforms such as Twitter, TalkLife, and Reddit are extremely popular to e

## **6. Analisis Hubungan Antar Dokumen (Cosine Similarity & Graph)**

In [6]:
print("\n" + "="*50 + "\n🔗 Memulai TAHAP 2: Analisis Hubungan Antar Dokumen...")

# Variabel 'embeddings_map' dan 'filenames_list' SUDAH TERISI dari TAHAP 1
valid_embeddings_doc = [embeddings_map[fname] for fname in filenames_list if embeddings_map.get(fname) is not None]
valid_filenames_doc = [fname for fname in filenames_list if embeddings_map.get(fname) is not None]

if len(valid_embeddings_doc) >= 2:
    # (Salin semua kode dari Cell 6 Anda, mulai dari 'embedding_matrix = ...')
    embedding_matrix_doc = np.array(valid_embeddings_doc)
    print(f"📐 Menghitung Cosine Similarity untuk {len(valid_filenames_doc)} dokumen...")
    similarity_matrix_doc = cosine_similarity(embedding_matrix_doc)
    print(f"✅ Matriks Kemiripan ({similarity_matrix_doc.shape}) selesai.")
    print("\n--- Matriks Kemiripan (Nilai Sebenarnya) ---")
    print(similarity_matrix_doc.round(3))

    nodes_doc = []
    edges_doc = []
    print(f"\n📈 Membangun data graf (Threshold > {SIMILARITY_THRESHOLD})...")
    for i in range(len(valid_filenames_doc)):
        # (Salin semua kode 'for i in range...' Anda dari Cell 6)
        fname_i = valid_filenames_doc[i]
        nodes_doc.append({"id": fname_i, "label": fname_i[:25]+"..."})
        for j in range(i + 1, len(valid_filenames_doc)):
            fname_j = valid_filenames_doc[j]
            similarity_score_doc = similarity_matrix_doc[i][j]
            if similarity_score_doc > SIMILARITY_THRESHOLD:
                edges_doc.append({
                    "from": fname_i, "to": fname_j,
                    "value": float(similarity_score_doc),
                    "title": f"Similarity: {similarity_score_doc:.3f}"
                })

    print(f"📊 Data Graf Siap: {len(nodes_doc)} nodes, {len(edges_doc)} edges.")

    # (Salin kode penyimpanan 'graph_data' Anda dari Cell 6)
    graph_data = {"nodes": nodes_doc, "edges": edges_doc}
    print(f"\n💾 Menyimpan data graf ke: {GRAPH_JSON_PATH}")
    try:
        with open(GRAPH_JSON_PATH, 'w') as f:
            json.dump(graph_data, f, indent=2)
        print(f"✅ Data graf berhasil disimpan.")
    except Exception as e:
        print(f"❌ Gagal menyimpan data graf: {e}")

    if edges_doc:
        # (Salin kode 'sorted_edges' Anda dari Cell 6)
        sorted_edges_doc = sorted(edges_doc, key=lambda x: x['value'], reverse=True)
        print("\n✨ Contoh 5 Hubungan Terkuat:")
        for edge in sorted_edges_doc[:5]:
             print(f"  - {edge['from'][:30]}... <-> {edge['to'][:30]}... (Sim: {edge['value']:.3f})")
    else:
        print("ℹ️ Tidak ditemukan hubungan signifikan di atas threshold.")
else:
    print("⚠️ Jumlah dokumen dengan embedding valid kurang dari 2. Analisis hubungan tidak dapat dilakukan.")

print("\n" + "="*50 + "\n✅ TAHAP 2 Selesai!")


🔗 Memulai TAHAP 2: Analisis Hubungan Antar Dokumen...
📐 Menghitung Cosine Similarity untuk 5 dokumen...
✅ Matriks Kemiripan ((5, 5)) selesai.

--- Matriks Kemiripan (Nilai Sebenarnya) ---
[[1.    0.102 0.246 0.378 0.235]
 [0.102 1.    0.514 0.528 0.45 ]
 [0.246 0.514 1.    0.526 0.488]
 [0.378 0.528 0.526 1.    0.507]
 [0.235 0.45  0.488 0.507 1.   ]]

📈 Membangun data graf (Threshold > 0.5)...
📊 Data Graf Siap: 5 nodes, 4 edges.

💾 Menyimpan data graf ke: /content/drive/MyDrive/PaperTA_graph_data.json
✅ Data graf berhasil disimpan.

✨ Contoh 5 Hubungan Terkuat:
  - Classification of mental illne... <-> A Multi-Class Deep Learning Ap... (Sim: 0.528)
  - A Computational Approach to Un... <-> A Multi-Class Deep Learning Ap... (Sim: 0.526)
  - Classification of mental illne... <-> A Computational Approach to Un... (Sim: 0.514)
  - A Multi-Class Deep Learning Ap... <-> AI in mental health.pdf... (Sim: 0.507)

✅ TAHAP 2 Selesai!


No 7

In [7]:
print("\n" + "="*50 + "\n💡 Memulai TAHAP 3: Inovasi 2 - Identifikasi Celah Riset...")

# Variabel 'embeddings_map', 'filenames_list', dan 'results' SUDAH TERISI
valid_embeddings_inno2 = [embeddings_map[fname] for fname in filenames_list if embeddings_map.get(fname) is not None]
valid_filenames_inno2 = [fname for fname in filenames_list if embeddings_map.get(fname) is not None]

if len(valid_embeddings_inno2) < 3:
    print(f"⚠️ Jumlah dokumen dengan embedding valid ({len(valid_embeddings_inno2)}) terlalu sedikit untuk analisis kluster yang bermakna.")
else:
    # (Salin semua kode dari Cell 7 Anda, mulai dari 'embedding_matrix_inno2 = ...')
    print(f"✅ Menemukan {len(valid_embeddings_inno2)} dokumen dengan embedding valid untuk dianalisis klusternya.")
    embedding_matrix_inno2 = np.array(valid_embeddings_inno2)

    num_documents_inno2 = embedding_matrix_inno2.shape[0]
    n_clusters = max(2, min(5, num_documents_inno2 // 2))
    print(f"\n⏳ Melakukan klasterisasi K-Means dengan k={n_clusters}...")
    start_cluster_time = time.time()
    try:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(embedding_matrix_inno2)
        end_cluster_time = time.time()
        print(f"✅ Klasterisasi selesai ({end_cluster_time - start_cluster_time:.2f} dtk).")

        # (Salin semua kode analisis kluster dan saran celah riset Anda dari Cell 7)
        print("\n📊 Menganalisis konten kluster berdasarkan kata kunci...")
        clusters = {i: [] for i in range(n_clusters)}
        cluster_keywords = {i: Counter() for i in range(n_clusters)}

        for filename, label in zip(valid_filenames_inno2, cluster_labels):
            clusters[label].append(filename)
            if filename in results and 'keywords' in results[filename]:
                cluster_keywords[label].update(results[filename]['keywords'])

        print("\n--- Hasil Klasterisasi Dokumen ---")
        for i in range(n_clusters):
            print(f"\n  Cluster {i}:")
            print(f"    Dokumen ({len(clusters[i])}):")
            for fname in clusters[i]:
                 print(f"      - {fname}")
            top_5_keywords = cluster_keywords[i].most_common(5)
            print(f"    Top Keywords: {[kw for kw, count in top_5_keywords]}")

        print("\n\n💡 Mencari Potensi Celah Riset (Area Persimpangan)...")
        suggested_gaps = []
        for i, j in itertools.combinations(range(n_clusters), 2):
            keywords_i = [kw for kw, count in cluster_keywords[i].most_common(3)]
            keywords_j = [kw for kw, count in cluster_keywords[j].most_common(3)]
            if keywords_i and keywords_j:
                 suggestion1 = f"Kombinasi antara '{keywords_i[0]}' (dari Cluster {i}) dan '{keywords_j[0]}' (dari Cluster {j})"
                 suggested_gaps.append(suggestion1)
                 if len(keywords_i) > 1 and len(keywords_j) > 1:
                      suggestion2 = f"Persimpangan area '{keywords_i[1]}' (Cluster {i}) dan '{keywords_j[1]}' (Cluster {j})"
                      suggested_gaps.append(suggestion2)

        if suggested_gaps:
            print("\n--- Saran Area Potensial (Berdasarkan Koleksi Anda Saat Ini) ---")
            for idx, gap in enumerate(suggested_gaps[:5]):
                 print(f"  {idx+1}. {gap}")
            print("\n(Saran ini bersifat heuristik, perlu validasi lebih lanjut)")
        else:
            print("ℹ️ Tidak dapat menghasilkan saran celah riset otomatis dari kluster yang ada.")

    except Exception as cluster_error:
        print(f"❌ Error saat melakukan klasterisasi atau analisis: {cluster_error}")

print("\n" + "="*50 + "\n✅ TAHAP 3 (Inovasi 2) Selesai!")


💡 Memulai TAHAP 3: Inovasi 2 - Identifikasi Celah Riset...
✅ Menemukan 5 dokumen dengan embedding valid untuk dianalisis klusternya.

⏳ Melakukan klasterisasi K-Means dengan k=2...
✅ Klasterisasi selesai (0.06 dtk).

📊 Menganalisis konten kluster berdasarkan kata kunci...

--- Hasil Klasterisasi Dokumen ---

  Cluster 0:
    Dokumen (4):
      - Classification of mental illnesses on social media using RoBERTa.pdf
      - A Computational Approach to Understand Mental Health from RedditKnowledge-Aware Multitask Learning Framework.pdf
      - A Multi-Class Deep Learning Approach for Early Detection of.pdf
      - AI in mental health.pdf
    Top Keywords: ['pandemic', 'work pandemic', 'social mental', 'anxiety bipolar', 'anxiety world']

  Cluster 1:
    Dokumen (1):
      - Comprehensive Comparative Study of.pdf
    Top Keywords: ['predictive performance', 'labels predicted', 'mlc taxonomy', 'class classification', 'methods datasets']


💡 Mencari Potensi Celah Riset (Area Persimpangan)..

In [8]:
# import itertools
# import nltk # Pastikan diimpor di sini
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# import time
# import json
# import os

# print("\n" + "="*50 + "\n💡 Memulai Inovasi 1: Pelacakan Argumen Lintas Dokumen (via Kalimat)...")

# # --- LANGKAH KRUSIAL: PASTIKAN PUNKT TERSEDIA DI SEL INI ---
# try:
#     print("Memverifikasi data NLTK (punkt)...")
#     # Check for both 'punkt' and 'punkt_tab'
#     nltk.data.find('tokenizers/punkt')
#     nltk.data.find('tokenizers/punkt_tab') # Check for punkt_tab specifically
#     print("✅ Data NLTK (punkt dan punkt_tab) sudah tersedia.")
# except LookupError:
#     print("⚠️ Data NLTK (punkt atau punkt_tab) tidak ditemukan di lokasi pencarian, mencoba mengunduh...")
#     try:
#         nltk.download('punkt', quiet=True) # Unduh tanpa banyak output
#         nltk.download('punkt_tab', quiet=True) # Unduh punkt_tab juga
#         # Check again after download
#         nltk.data.find('tokenizers/punkt')
#         nltk.data.find('tokenizers/punkt_tab')
#         print("✅ Data NLTK (punkt dan punkt_tab) berhasil diunduh dan diverifikasi.")
#     except Exception as download_error:
#         print(f"❌ FATAL ERROR: Gagal mengunduh data NLTK (punkt atau punkt_tab). Error: {download_error}")
#         print("Pastikan koneksi internet Colab stabil.")
#         # Hentikan eksekusi sel ini jika download gagal
#         raise download_error
# # -----------------------------------------------------------------

# # --- Periksa apakah variabel yang dibutuhkan ada ---
# required_vars = ['document_texts', 'embedding_model', 'SENTENCE_SIMILARITY_THRESHOLD', 'SENTENCE_RELATIONS_JSON_PATH']
# vars_exist = True
# for var in required_vars:
#     if var not in globals():
#         print(f"❌ FATAL ERROR: Variabel '{var}' belum terdefinisi. Pastikan sel sebelumnya (terutama Cell 3, 4, 5) sudah dijalankan.")
#         vars_exist = False
# if not vars_exist:
#     raise NameError("Variabel penting tidak ditemukan.")
# elif not document_texts:
#     print("⚠️ Peringatan: Variabel 'document_texts' kosong. Cell 5 mungkin belum berhasil mengekstrak teks.")
#     # Kita tetap lanjutkan, tapi kemungkinan besar tidak akan ada hasil
# else:
#      print(f"✅ Menemukan {len(document_texts)} teks dokumen untuk dianalisis kalimatnya.")


# # --- Langkah 1: Ekstrak Kalimat & Buat Embeddings ---
# all_sentences_data = []
# total_sentences_processed = 0
# sentence_embedding_start_time = time.time()

# print("\n⏳ Mengekstrak dan membuat embedding untuk semua kalimat...")
# if document_texts: # Hanya proses jika ada teks
#     for filename, text in document_texts.items():
#         if not text: continue

#         # --- Panggil split_into_sentences (SEKARANG SEHARUSNYA BERHASIL) ---
#         # Fungsi ini didefinisikan di Cell 3 dan sudah dimodifikasi untuk memaksa download
#         # Namun, error 'punkt_tab' muncul di sini, jadi kita tambahkan download di sel ini juga.
#         sentences = split_into_sentences(text)
#         if not sentences:
#             print(f"  ⚠️ Tidak ada kalimat valid ditemukan di {filename}.")
#             continue

#         print(f"  - Memproses {len(sentences)} kalimat dari {filename}...")
#         try:
#             sentence_embeddings = embedding_model.encode(sentences, convert_to_numpy=True, show_progress_bar=False)
#             for i, (sentence, embedding) in enumerate(zip(sentences, sentence_embeddings)):
#                  # Pastikan embedding adalah numpy array sebelum menambahkannya
#                 if embedding is not None and isinstance(embedding, np.ndarray):
#                     all_sentences_data.append((filename, i, sentence, embedding))
#                     total_sentences_processed += 1
#                 else:
#                     print(f"  ⚠️ Skipping sentence {i} in {filename} due to invalid embedding.")
#         except Exception as e:
#             print(f"  ❌ Error saat membuat embedding kalimat untuk {filename}: {e}")
# else:
#     print("ℹ️ Tidak ada teks dokumen untuk diproses kalimatnya.")


# sentence_embedding_end_time = time.time()
# print(f"\n✅ Selesai membuat embedding untuk {total_sentences_processed:,} kalimat.")
# print(f"⏱️ Waktu Embedding Kalimat: {sentence_embedding_end_time - sentence_embedding_start_time:.2f} detik")

# # --- Langkah 2: Hitung Kemiripan Antar Kalimat ---
# sentence_relationships = []
# similarity_calculation_start_time = time.time()

# print(f"\n⏳ Menghitung kemiripan antar kalimat (Threshold > {SENTENCE_SIMILARITY_THRESHOLD})...")

# # Filter data hanya jika embedding berhasil dibuat dan berupa numpy array
# valid_sentence_data = [data for data in all_sentences_data if data[3] is not None and isinstance(data[3], np.ndarray)]

# if len(valid_sentence_data) >= 2:
#     try:
#         all_embeddings = np.array([data[3] for data in valid_sentence_data])
#         all_metadata = [(data[0], data[1], data[2]) for data in valid_sentence_data] # (filename, sent_idx, text)

#         print(f"  - Menghitung matriks similarity untuk {all_embeddings.shape[0]} kalimat...")
#         sentence_similarity_matrix = cosine_similarity(all_embeddings)
#         print(f"  - Matriks similarity ({sentence_similarity_matrix.shape}) selesai.")

#         # (Loop for i in range... untuk mencari pasangan kalimat tetap sama)
#         num_pairs_found = 0
#         for i in range(len(all_metadata)):
#             for j in range(i + 1, len(all_metadata)):
#                 if all_metadata[i][0] != all_metadata[j][0]:
#                      similarity_score = sentence_similarity_matrix[i, j]
#                      if similarity_score > SENTENCE_SIMILARITY_THRESHOLD:
#                           num_pairs_found += 1
#                           sentence_relationships.append({
#                               "file1": all_metadata[i][0], "sent1_idx": all_metadata[i][1], "sent1_text": all_metadata[i][2],
#                               "file2": all_metadata[j][0], "sent2_idx": all_metadata[j][1], "sent2_text": all_metadata[j][2],
#                               "similarity": float(similarity_score)
#                           })

#         # (Kode penyimpanan JSON dan print hasil tetap sama)
#         similarity_calculation_end_time = time.time()
#         print(f"✅ Ditemukan {num_pairs_found:,} pasangan kalimat signifikan.")
#         print(f"⏱️ Waktu Perhitungan Similarity Kalimat: {similarity_calculation_end_time - similarity_calculation_start_time:.2f} detik")

#         print(f"\n💾 Menyimpan hasil hubungan kalimat ke: {SENTENCE_RELATIONS_JSON_PATH}")
#         try:
#             sentence_relationships.sort(key=lambda x: x['similarity'], reverse=True)
#             with open(SENTENCE_RELATIONS_JSON_PATH, 'w') as f:
#                 json.dump(sentence_relationships, f, indent=2)
#             print(f"✅ Hasil hubungan kalimat berhasil disimpan.")

#             print("\n✨ Contoh 5 Hubungan Kalimat Terkuat:")
#             for rel in sentence_relationships[:5]:
#                  print(f"  - [{rel['file1'][:20]}... Sent {rel['sent1_idx']}]")
#                  print(f"    '{rel['sent1_text'][:100]}...'")
#                  print(f"  - [{rel['file2'][:20]}... Sent {rel['sent2_idx']}]")
#                  print(f"    '{rel['sent2_text'][:100]}...'")
#                  print(f"    (Similarity: {rel['similarity']:.3f})\n")

#         except Exception as e:
#             print(f"❌ Gagal menyimpan hasil hubungan kalimat: {e}")

#     except Exception as matrix_error:
#         print(f"❌ Error saat menghitung atau memproses matriks similarity: {matrix_error}")

# else:
#     print("⚠️ Jumlah kalimat dengan embedding valid kurang dari 2. Analisis hubungan tidak dapat dilakukan.")

# print("\n" + "="*50 + "\n✅ Inovasi 1 Selesai!")